In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [2]:
data_dir = 'C:\\Users\\rampa\\Downloads\\archive\\train'

In [5]:
from tensorflow.keras.utils import image_dataset_from_directory
#train_data=image_dataset_from_directory(data_dir)
image_height = 256
batch_size=32
train_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=3,
  image_size=(image_height, image_height),
  batch_size=batch_size)

Found 2356 files belonging to 28 classes.
Using 1885 files for training.


In [7]:
class_names = train_ds.class_names

len(class_names)

28

In [9]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [11]:
val_ds = image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=3,
  image_size=(image_height, image_height),
  batch_size=batch_size)

Found 2356 files belonging to 28 classes.
Using 471 files for validation.


In [13]:
from keras.applications import ResNet50
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Activation, Flatten, Dense

resnet_model = Sequential()

pretrained_model= ResNet50(include_top=False,
                   input_shape=(256,256,3),
                   pooling='avg',
                   classes=28,
                   weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(28, activation='softmax'))

In [15]:
from keras.optimizers import Adam
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [17]:
resnet_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 28)             │        14,364 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,651,164 (94.04 MB)

 Trainable params: 1,063,452 (4.06 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
epochs=5
history = resnet_model.fit(
  train_ds,
  validation_data = val_ds,  
  epochs=epochs
)

Epoch 1/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 353s 6s/step - accuracy: 0.2382 - loss: 3.0469 - val_accuracy: 0.4183 - val_loss: 1.9192
Epoch 2/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 303s 5s/step - accuracy: 0.6339 - loss: 1.1970 - val_accuracy: 0.5202 - val_loss: 1.6395
Epoch 3/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 270s 5s/step - accuracy: 0.7470 - loss: 0.8060 - val_accuracy: 0.5520 - val_loss: 1.4299
Epoch 4/100
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8457 - loss: 0.5108